In [1]:
import luigi
import requests
import json
import pandas as pd
import datetime

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Как-может-выглядеть-пайплайн---не-самый-лучший-вариант" data-toc-modified-id="Как-может-выглядеть-пайплайн---не-самый-лучший-вариант-0.0.1"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>Как может выглядеть пайплайн - не самый лучший вариант</a></span></li><li><span><a href="#Luigi" data-toc-modified-id="Luigi-0.0.2"><span class="toc-item-num">0.0.2&nbsp;&nbsp;</span>Luigi</a></span><ul class="toc-item"><li><span><a href="#Запуск-Luigi" data-toc-modified-id="Запуск-Luigi-0.0.2.1"><span class="toc-item-num">0.0.2.1&nbsp;&nbsp;</span>Запуск Luigi</a></span></li><li><span><a href="#Пример-класса-Task" data-toc-modified-id="Пример-класса-Task-0.0.2.2"><span class="toc-item-num">0.0.2.2&nbsp;&nbsp;</span>Пример класса Task</a></span></li><li><span><a href="#Target" data-toc-modified-id="Target-0.0.2.3"><span class="toc-item-num">0.0.2.3&nbsp;&nbsp;</span>Target</a></span></li><li><span><a href="#Parameters" data-toc-modified-id="Parameters-0.0.2.4"><span class="toc-item-num">0.0.2.4&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Зависимости" data-toc-modified-id="Зависимости-0.0.2.5"><span class="toc-item-num">0.0.2.5&nbsp;&nbsp;</span>Зависимости</a></span></li></ul></li><li><span><a href="#Пример-пайплайна-для--данных-osm" data-toc-modified-id="Пример-пайплайна-для--данных-osm-0.0.3"><span class="toc-item-num">0.0.3&nbsp;&nbsp;</span>Пример пайплайна для  данных osm</a></span><ul class="toc-item"><li><span><a href="#Плюсы-Luigi" data-toc-modified-id="Плюсы-Luigi-0.0.3.1"><span class="toc-item-num">0.0.3.1&nbsp;&nbsp;</span>Плюсы Luigi</a></span></li></ul></li><li><span><a href="#Ограничения-Luigi" data-toc-modified-id="Ограничения-Luigi-0.0.4"><span class="toc-item-num">0.0.4&nbsp;&nbsp;</span>Ограничения Luigi</a></span></li><li><span><a href="#Другие-библиотеки" data-toc-modified-id="Другие-библиотеки-0.0.5"><span class="toc-item-num">0.0.5&nbsp;&nbsp;</span>Другие библиотеки</a></span></li></ul></li></ul></li></ul></div>

### Как может выглядеть пайплайн - не самый лучший вариант

In [2]:
def load_data():
    pass

def process_data():
    pass

def log_results():
    pass

def train_model():
    pass

def predict_():
    pass

def store_results():
    pass

if __name__ == "__main__":
    load_data()
    process_data()
    log_results()
    train_model()
    predict_()
    store_results()

Можно унифицировать подход к созданию пайплайна - использовать единую структуру для разных пайплайнов

### Luigi

Luigi позволяет создавать последовательные задачи обработки данных и определять зависимости между ними.Пайплайн представляет из себя направленный ациклический граф. Luigi поддерживает разные источники данных - это могут локальные файлы, база данных (например MySQL, Mongodb), hdfs. 

Документация luigi https://luigi.readthedocs.io/

Установка luigi `pip3 install luigi`

`Luigi` оперирует тремя понятиями - `Task`, `Target`, `Parameter`.

`Task` - класс, отвечает за саму задачу. например, здесь можно писать код обучения модели или обработки данных.

`Target` - отвечает за то, куда сохранять и откуда читать данные. Это может быть и база данных, и хранение на локальном диске, или hdfs.

`Parameter` - параметры, с которым запускается задача - например дата запуска. Параметры могут быть опциональными.

#### Запуск Luigi 

Нужно запустить `luigid` в терминале,далее можно посмотреть интерфейс можно на http://localhost:8082

<img src=luigi_interface.png>

#### Пример класса Task

Для создания класса Task мы наследуем luigi.Task и нам нужно определить три основных метода: `run`, `output`, `requires`.
Создадим простую задачу, где принимаемая на вход строка записывается в текстовый файл. Также у класса luigi.Task есть метод `input`, но его переопределять не нужно, он служит некоторой оберткой над `requires` и возвращает Target зависимых задач. Рассмотрим пример:

In [5]:
class MyTask(luigi.Task):
    
    def output(self):
        return luigi.LocalTarget(path="output.txt")
    
    def run(self):
        with self.output().open('w') as f:
            f.write("build luigi pipeline")   

Запуск пайплайна происходит следующим образом:

In [8]:
if __name__ == '__main__':
    luigi.build([MyTask()])

DEBUG: Checking if MyTask() is complete
NoneType: None
INFO: Retrying attempt 2 of 3 (max)
INFO: Wait for 30 seconds
NoneType: None
INFO: Retrying attempt 2 of 3 (max)
INFO: Wait for 30 seconds
NoneType: None
INFO: Retrying attempt 3 of 3 (max)
INFO: Wait for 30 seconds
NoneType: None
INFO: Retrying attempt 4 of 3 (max)
INFO: Wait for 30 seconds
INFO: Worker Worker(salt=185470664, workers=1, host=MacBook-Pro-Ildar.local, username=ildar, pid=16110) was stopped. Shutting down Keep-Alive thread


KeyboardInterrupt: 

#### Target

LocalTarget

In [ ]:
def output(self):
        return luigi.LocalTarget(path='output.csv')

Database target

In [ ]:
from luigi.contrib.mongodb import MongoTarget
from pymongo import MongoClient

def output(self):
    
    connection = MongoClient(self.host, self.port)
    db_client = connection[self.db_name]
    collection_name = 'collection'

    return MongoTarget(db_client, '_id', collection_name)

HDFS target

In [ ]:
def output(self):
        return luigi.contrib.hdfs.HdfsTarget("data/logs_%s.csv" % self.date)

Если какая-нибудь задача из пайплана обрывается, то нужно перезапустить пайплайн с этой задачи - результаты предыдущих задач не удаляются (обеспечивается методом exists в классе Target).

#### Parameters

Для создания параметров используется класс `Parameter` - по умолчанию возвращается класс string.

Также для определения типа параметров можно использовать следующие классы - например `DateParameter`, `IntParameter`, `FloatParameter`. Другие типы параметров можно найти в документации.

Определение типов параметров нужно для того, чтобы при запуске пайплайна из командной строки конвертация параметров из string происходила в нужный тип.

In [ ]:
class Report(luigi.Task):
    
    path = luigi.Parameter()
    date = luigi.DateParameter(default=datetime.date.today())
    input_data = luigi.IntParameter()
    

In [ ]:
d = Report('/data', datetime.date(2012, 5, 10), 1)
print(d.path, d.date, d.input_data)

/data 2012-05-10 1


В Luigi Task Visualiser появится задача и ее статус:

#### Зависимости

Может возникать несколько зависимостей между тасками, для этого нужно определить метод `requires`, который возвращает другой таск, который должен исполниться до запуска основной задачи. 

Также метод `requires` может возвращать лист зависимых задач - мы посмотрим это позже на примере пайплайна для скачивания данных с OpenStreetMap.

In [ ]:
class MyTask(luigi.Task):
    
    def run(self):
        with self.output().open('w') as f:
            f.write('luigi pipeline')
        
    def output(self):
        return luigi.LocalTarget(path="output_one.txt")
    
class MyTaskModified(luigi.Task):
    
    string = luigi.Parameter()
    
    def requires(self):
        return MyTask()
    
    def output(self):
        luigi.LocalTarget(path='output_two.txt')
        
    def run(self):
        with self.input().open('r') as f:
            data = f.read()
            
        with self.input().open('w') as fout:
            fout.write(data + self.string)

In [ ]:
if __name__ == '__main__':
    luigi.build([MyTaskModified('test dependencies')])

DEBUG: Checking if MyTaskModified(string=test dependencies) is complete
/usr/local/lib/python3.5/site-packages/luigi/worker.py:401: UserWarning: Task MyTaskModified(string=test dependencies) without outputs has no custom complete() method
  is_complete = task.complete()
DEBUG: Checking if MyTask() is complete
INFO: Informed scheduler that task   MyTaskModified_test_dependencie_f8899dc010   has status   PENDING
INFO: Informed scheduler that task   MyTask__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 2
INFO: [pid 7509] Worker Worker(salt=411079994, workers=1, host=MacBook-Air-Elena-2.local, username=elena, pid=7509) running   MyTask()
INFO: [pid 7509] Worker Worker(salt=411079994, workers=1, host=MacBook-Air-Elena-2.local, username=elena, pid=7509) done      MyTask()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   MyTask__99914b932b

In [ ]:
import luigi
import requests
import json
import pandas as pd
import datetime

### Пример пайплайна для  данных osm

Скачать данные по выбранной категории объектов

Обработка данных

Сохранение для дальнейшего использлвания

Основными параметрами данного пайплайна будет территория, заданная ббоксом города, категория места, а также атрибут OSM.
Скачаем места по Москве:

In [ ]:
class TaskOSMLoading(luigi.Task):
    
    attr = luigi.Parameter()
    category = luigi.Parameter()
    bbox = luigi.Parameter()
    
    def output(self):
        return luigi.LocalTarget("osm_data/data_osm_{}.json".format(self.category))
        
    def run(self):
        
        overpass_url = "https://overpass.kumi.systems/api/interpreter"
        overpass_query = """
            [out:json];
            ({}[{}]({});
            );
             out body;
                         """.format(self.attr, self.category, self.bbox)
        
        response = requests.get(overpass_url, 
                                params={'data': overpass_query})
        data = response.json()
        
        with self.output().open('w') as f:
            json.dump(data, f)

В классе можно определять вспомогательные методы, которые затем можно переиспользовать в `run()`:

In [ ]:
class TaskOSMProcessing(luigi.Task):
    
    attr = luigi.Parameter()
    categories = luigi.ListParameter()
    bbox = luigi.Parameter()
    #date = luigi.DateParameter(default = datetime.datetime.now()) 
    
    def requires(self):
        return [TaskOSMLoading(self.attr, category, self.bbox) for category in self.categories]
    
    def output(self):
        return luigi.LocalTarget('osm_data/results_s.csv')
    
    def get_tag(self, x, key_name):
        if str(key_name) in x:
            tmp = x[key_name]
            return tmp
        else:
            return 'unknown'
    
    def run(self):
        
        all_osm = pd.DataFrame()
        for _input in self.input():
            with _input.open('r') as raw_file:
                data = pd.DataFrame((json.load(raw_file))['elements'])
                all_osm = all_osm.append(data)
        
        all_osm['amenity'] = all_osm['tags'].apply(lambda x: self.get_tag(x, 'amenity'))
        
        with self.output().open('w') as f:
            all_osm[['id','lat','lon','amenity']].to_csv(f, encoding='utf8')

In [ ]:
if __name__ == '__main__':
    luigi.build([TaskOSMProcessing('node',['amenity','shop','office','bus_stop'],
                                   "55.1422,36.8031,56.0212,37.9674")])

DEBUG: Checking if TaskOSMProcessing(attr=node, categories=["amenity", "shop", "office", "bus_stop"], bbox=55.1422,36.8031,56.0212,37.9674) is complete
DEBUG: Checking if TaskOSMLoading(attr=node, category=amenity, bbox=55.1422,36.8031,56.0212,37.9674) is complete
DEBUG: Checking if TaskOSMLoading(attr=node, category=shop, bbox=55.1422,36.8031,56.0212,37.9674) is complete
DEBUG: Checking if TaskOSMLoading(attr=node, category=office, bbox=55.1422,36.8031,56.0212,37.9674) is complete
DEBUG: Checking if TaskOSMLoading(attr=node, category=bus_stop, bbox=55.1422,36.8031,56.0212,37.9674) is complete
INFO: Informed scheduler that task   TaskOSMProcessing_node_55_1422_36_8031____amenity____sho_badb57d548   has status   PENDING
INFO: Informed scheduler that task   TaskOSMLoading_node_55_1422_36_8031__bus_stop_cc6b2eb261   has status   PENDING
INFO: Informed scheduler that task   TaskOSMLoading_node_55_1422_36_8031__office_a8ea274f7d   has status   PENDING
INFO: Informed scheduler that task   Ta

<img src=luigi_dependencies.png>

#### Плюсы Luigi 

Luigi можно запускать для тестирования пайплайна в режиме `--local-sheduler`, также можно запускать через центральный планировщик с аргументом `--central-sheduler`  

Довольно простой интерфейс - по факту для написания пайплайна нужно оперировать только тремя сущностями;

Неплохая документация;

Визуализация графа задач;

Перезапуск задачи не приводит к потере данных предыдущих задач;

Удобная обработка аргументов, передаваемые через терминал;

### Ограничения Luigi

Не поддерживает real-time обработку

Отсутствует запуск задач по расписанию

### Другие библиотеки

airflow - библиотека от airbnb, сейчас выложена в открытом доступе

celery - подходит для real-time обработки